In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [6]:
# Define the path where you want to store the dataset
dataset_folder_path = '/content/drive/MyDrive/Datasets'
cifar100_dataset_folder_path = os.path.join(dataset_folder_path, 'cifar100')


In [12]:
import kagglehub
import os
import shutil


# Create the directory if it doesn't exist
os.makedirs(cifar100_dataset_folder_path, exist_ok=True)

# Download the dataset
print("Downloading dataset...")
path = kagglehub.dataset_download("ibraheemmoosa/cifar100-256x256")
print("Path to dataset files:", path)

# Move the downloaded files to the desired directory
for file_name in os.listdir(path):
    source_file_path = os.path.join(path, file_name)
    destination_file_path = os.path.join(cifar100_dataset_folder_path, file_name)

    # Move the file
    shutil.move(source_file_path, destination_file_path)

print("Dataset has been moved to:", cifar100_dataset_folder_path)

# Check if the dataset is nested in a subdirectory (e.g., 'cifar100-preprocessed')
nested_folder = os.path.join(cifar100_dataset_folder_path, 'cifar100-preprocessed')
if os.path.exists(nested_folder):
    print("Found nested dataset folder. Moving contents to the root directory...")
    for item in os.listdir(nested_folder):
        source_item_path = os.path.join(nested_folder, item)
        destination_item_path = os.path.join(cifar100_dataset_folder_path, item)

        # Move the item
        shutil.move(source_item_path, destination_item_path)

    # Remove the now-empty nested folder
    os.rmdir(nested_folder)
    print("Contents moved to the root directory.")

# Verify the contents of the target directory
print("Contents of the target directory:", os.listdir(cifar100_dataset_folder_path))

Path to dataset files: /root/.cache/kagglehub/datasets/ibraheemmoosa/cifar100-256x256/versions/1
Dataset has been moved to: /content/drive/MyDrive/Datasets/cifar100
Found nested dataset folder. Moving contents to the root directory...
Contents moved to the root directory.
Contents of the target directory: ['test', 'train']


In [20]:
import os
import re
import cv2
import numpy as np
import torch
from torch.utils.data import Dataset
from torchvision import transforms
from PIL import Image
import torch.optim as optim
from torch.optim.lr_scheduler import ReduceLROnPlateau
from torch.utils.data import DataLoader
import matplotlib.pyplot as plt

In [3]:
!nvidia-smi

Fri Jan 10 22:29:28 2025       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       Off | 00000000:00:04.0 Off |                    0 |
| N/A   31C    P8               9W /  70W |      0MiB / 15360MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [7]:
print(f'files in the dataset folder path{os.listdir(cifar100_dataset_folder_path)}')

files in the dataset folder path['test', 'train']


In [8]:

# Define the dataset path
cifar100_dataset_folder_path = '/content/drive/MyDrive/Datasets/cifar100'

# List files in the train folder
train_folder = os.path.join(cifar100_dataset_folder_path, 'train')
train_files = os.listdir(train_folder)
print("Files in 'train' folder:", len(train_files))

# List files in the test folder
test_folder = os.path.join(cifar100_dataset_folder_path, 'test')
test_files = os.listdir(test_folder)
print("Files in 'test' folder:", len(test_files))

Files in 'train' folder: 50000
Files in 'test' folder: 10000


In [9]:


class CustomCIFAR100Dataset(Dataset):
    def __init__(self, root_dir, transform=None):
        """
        Args:
            root_dir (str): Path to the dataset folder (e.g., '/content/drive/MyDrive/Datasets/cifar100/train').
            transform (callable, optional): Optional transform to be applied to the images.
        """
        self.root_dir = root_dir
        self.transform = transform

        # Get all image file paths in the root directory
        self.image_paths = [os.path.join(root_dir, fname) for fname in os.listdir(root_dir) if fname.endswith('.png')]

        # Ensure images are found
        if not self.image_paths:
            raise RuntimeError(f"No images found in {root_dir}. Make sure the folder contains .png files.")

        # Extract labels from filenames
        self.labels = self._extract_labels_from_filenames()

    def _extract_labels_from_filenames(self):
        """
        Extracts labels from filenames using a regular expression.
        Assumes filenames contain a 2-digit number representing the class label.
        Example: 'image_23.png' -> label = 23.
        """
        labels = []
        for path in self.image_paths:
            # Extract the filename from the full path
            filename = os.path.basename(path)

            # Use a regular expression to find the 2-digit label in the filename
            match = re.search(r'\d{2}', filename)
            if not match:
                raise ValueError(f"Filename {filename} does not contain a valid 2-digit label.")

            # Extract the label and convert it to an integer
            label = int(match.group())
            labels.append(label)

        return labels

    def __len__(self):
        """
        Returns the total number of images in the dataset.
        """
        return len(self.image_paths)

    def __getitem__(self, idx):
        """
        Loads and returns an image and its corresponding label.
        Args:
            idx (int): Index of the image to retrieve.
        Returns:
            image (torch.Tensor): The transformed image.
            label (int): The corresponding label.
        """
        # Get the image path and label
        image_path = self.image_paths[idx]
        label = self.labels[idx]

        # Load the image using OpenCV
        image = cv2.imread(image_path, cv2.IMREAD_COLOR)
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)  # Convert from BGR to RGB

        # Convert the image to a PIL image (required for torchvision.transforms)
        image = Image.fromarray(image)

        # Apply transformations (if any)
        if self.transform:
            image = self.transform(image)  # Apply torchvision.transforms

        # Convert the label to a PyTorch tensor
        label = torch.tensor(label, dtype=torch.int64)

        return image, label

In [10]:
from torchvision import transforms

# Define transformations
transform = transforms.Compose([
    transforms.Resize((256, 256)),  # Resize images to 256x256
    transforms.ToTensor(),  # Convert images to PyTorch tensors
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))  # Normalize to [-1, 1]
])

In [26]:
# Define paths
train_folder = '/content/drive/MyDrive/Datasets/cifar100/train'
test_folder = '/content/drive/MyDrive/Datasets/cifar100/test'

# Create dataset instances
train_dataset = CustomCIFAR100Dataset(root_dir=train_folder, transform=transform)
test_dataset = CustomCIFAR100Dataset(root_dir=test_folder, transform=transform)

In [27]:
from torch.utils.data import random_split

# Define the sizes for training and validation sets
train_size = int(0.8 * len(train_dataset))  # 80% for training
val_size = len(train_dataset) - train_size  # 20% for validation

# Split the training dataset
train_dataset, val_dataset = random_split(train_dataset, [train_size, val_size])

# Print the sizes of the splits
print(f"Training set size: {len(train_dataset)}")
print(f"Validation set size: {len(val_dataset)}")

Training set size: 40000
Validation set size: 10000


In [28]:
from torch.utils.data import DataLoader

# Define batch size
batch_size = 64

# Create data loaders
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)  # No need to shuffle validation data
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

# Check the number of batches
print("Number of training batches:", len(train_loader))
print("Number of validation batches:", len(val_loader))
print("Number of testing batches:", len(test_loader))

Number of training batches: 625
Number of validation batches: 157
Number of testing batches: 157


In [15]:
# Get a batch of data
images, labels = next(iter(train_loader))

# Check the shape of the batch
print("Batch images shape:", images.shape)  # Should be [batch_size, channels, height, width]
print("Batch labels shape:", labels.shape)  # Should be [batch_size]

Batch images shape: torch.Size([64, 3, 256, 256])
Batch labels shape: torch.Size([64])


#Resnet 18 model

In [29]:
# Initialize variables to save the best model
best_val_loss = float('inf')  # Start with infinity
best_model_weights = None

# Lists to store metrics
train_losses = []
val_losses = []
train_accuracies = []
val_accuracies = []

# Training function
def train(model, dataloader, criterion, optimizer, device):
    model.train()  # Set model to training mode
    running_loss = 0.0
    correct = 0
    total = 0

    for images, labels in dataloader:
        images, labels = images.to(device), labels.to(device)

        # Forward pass
        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        running_loss += loss.item()

        # Backward pass and optimization
        loss.backward()
        optimizer.step()

        # Calculate accuracy
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

    epoch_loss = running_loss / len(dataloader)
    epoch_accuracy = 100 * correct / total
    return epoch_loss, epoch_accuracy

# Validation function
def validate(model, dataloader, criterion, device):
    model.eval()  # Set model to evaluation mode
    running_loss = 0.0
    correct = 0
    total = 0

    with torch.no_grad():  # Disable gradient calculation
        for images, labels in dataloader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            loss = criterion(outputs, labels)
            running_loss += loss.item()

            # Calculate accuracy
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    epoch_loss = running_loss / len(dataloader)
    epoch_accuracy = 100 * correct / total
    return epoch_loss, epoch_accuracy



In [30]:
def test(model, dataloader, device):
    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for images, labels in dataloader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    accuracy = correct / total
    print(f"Test Accuracy: {accuracy:.4f}")



In [31]:
# Early Stopping Class
class EarlyStopping:
    def __init__(self, patience=5, verbose=False):
        self.patience = patience
        self.verbose = verbose
        self.counter = 0
        self.best_loss = float('inf')
        self.early_stop = False

    def __call__(self, val_loss):
        if val_loss < self.best_loss:
            self.best_loss = val_loss
            self.counter = 0  # Reset counter if validation loss improves
        else:
            self.counter += 1  # Increment counter if no improvement
            if self.counter >= self.patience:
                self.early_stop = True
                if self.verbose:
                    print("Early stopping triggered")

# Initialize early stopping
early_stopping = EarlyStopping(patience=5, verbose=True)


In [32]:
# Plot training and validation metrics
def plot_metrics(train_losses, val_losses, train_accuracies, val_accuracies):
    plt.figure(figsize=(12, 5))
    plt.subplot(1, 2, 1)
    plt.plot(train_losses, label='Train Loss')
    plt.plot(val_losses, label='Validation Loss')
    plt.xlabel('Epoch')
    plt.ylabel('Loss')
    plt.legend()

    plt.subplot(1, 2, 2)
    plt.plot(train_accuracies, label='Train Accuracy')
    plt.plot(val_accuracies, label='Validation Accuracy')
    plt.xlabel('Epoch')
    plt.ylabel('Accuracy')
    plt.legend()
    plt.show()


In [33]:
import torch.nn as nn
import torchvision.models as models

# Load a pre-trained ResNet-18 model
model = models.resnet18(pretrained=True)

# Modify the final layer for CIFAR-100 (100 classes)
model.fc = nn.Linear(model.fc.in_features, 100)

# Move the model to the appropriate device (CPU or GPU)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)

# Print the modified model architecture
print(model)

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
  

In [34]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)
scheduler = ReduceLROnPlateau(optimizer, mode='min', factor=0.4, patience=2, verbose=True)
num_epoch = 10

In [ ]:
# Training loop with early stopping and saving the best model
for epoch in range(num_epoch):
    # Train for one epoch
    train_loss, train_accuracy = train(model, train_loader, criterion, optimizer, device)
    train_losses.append(train_loss)
    train_accuracies.append(train_accuracy)

    # Validate for one epoch
    val_loss, val_accuracy = validate(model, val_loader, criterion, device)
    val_losses.append(val_loss)
    val_accuracies.append(val_accuracy)

    # Print epoch results
    print('-' * 100)
    print(f'Epoch [{epoch + 1}/{num_epoch}], '
          f'Train Loss: {train_loss:.4f}, Train Accuracy: {train_accuracy:.2f}%, '
          f'Val Loss: {val_loss:.4f}, Val Accuracy: {val_accuracy:.2f}%')

    # Step the scheduler based on validation loss
    scheduler.step(val_loss)

    # Check if the current validation loss is the best we've seen so far
    if val_loss < best_val_loss:
        best_val_loss = val_loss
        best_model_weights = model.state_dict()
        print("Best model weights saved.")

    # Check for early stopping
    early_stopping(val_loss)
    if early_stopping.early_stop:
        print(f"Early stopping triggered at epoch {epoch + 1}")
        break

# After training, save the best model weights to a file
torch.save(best_model_weights, 'best_model_weights.pth')
print("Best model weights saved to 'best_model_weights.pth'.")

In [ ]:

# Call this function at the end of training
plot_metrics(train_losses, val_losses, train_accuracies, val_accuracies)

In [ ]:
test(model, test_loader, device)

#YOLOv8 medium model

In [ ]:
!pip install ultralytics

In [ ]:
from ultralytics import YOLO

# Load a pretrained YOLOv8 classification model (medium size)
model = YOLO('yolov8m-cls.pt')  # 'm' stands for medium size

# Train the model on your dataset
results = model.train(
    data='/content/drive/MyDrive/Datasets/sports100',  # Path to your dataset
    epochs=10,                # Number of epochs
    imgsz=224,                # Image size
    batch=32,                 # Batch size
    name='yolov8m_cls_sports100',  # Name of the training run
    project='classification', # Project name
    optimizer='Adam',         # Optimizer
    lr0=0.001,               # Initial learning rate
    patience=3,              # Early stopping patience
    device='0'               # Use GPU (set to 'cpu' if no GPU is available)
)



In [ ]:
# Evaluate the model on the validation set
metrics = model.val()

# Export the model to ONNX format (optional)
model.export(format='onnx')